In [1]:
import json
import os
from datetime import datetime, timedelta
import pandas as pd
import gradio as gr

# --- Data Loading and Saving Functions ---
EQUIPOS_FILE = '/content/equipos.json'
MANTENIMIENTO_FILE = '/content/mantenimiento.json'
CALIBRACIONES_FILE = '/content/calibraciones.json'
ALERTS_FILE = 'alertas.json'

def cargar_datos():
    """Loads data from JSON files into dictionaries."""
    def load_json(filename):
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                try:
                    return json.load(f)
                except json.JSONDecodeError:
                    print(f"Warning: Could not decode JSON from {filename}. Returning empty dictionary.")
                    return {}
        else:
            return {}

    equipos = load_json(EQUIPOS_FILE)
    # Ensure 'NOMBRE' key exists for all equipment entries for robustness
    for eq_id in equipos:
        if 'NOMBRE' not in equipos[eq_id]:
            equipos[eq_id]['NOMBRE'] = 'Nombre Desconocido' # Provide a default name

    mantenimiento = load_json(MANTENIMIENTO_FILE)
    calibracion = load_json(CALIBRACIONES_FILE)
    alertas = load_json(ALERTS_FILE) # Load alerts data, though not used for saving/loading alerts in Tkinter code

    return equipos, mantenimiento, calibracion, alertas

def guardar_datos(equipos, mantenimiento, calibracion, alertas):
    """Saves data from dictionaries into JSON files."""
    def save_json(data, filename):
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)

    save_json(equipos, EQUIPOS_FILE)
    save_json(mantenimiento, MANTENIMIENTO_FILE)
    save_json(calibracion, CALIBRACIONES_FILE)
    save_json(alertas, ALERTS_FILE) # Save alerts data for consistency, though not used in Tkinter's saving logic

# --- GestorDatos Class (Core Logic) ---
class GestorDatos:
    """Encapsulates the core logic for managing equipment, maintenance, and calibration data."""
    def __init__(self, equipos, mantenimiento, calibracion, alertas):
        self.equipos = equipos
        self.mantenimiento = mantenimiento
        self.calibracion = calibracion
        self.alertas = alertas # alerts data is loaded but not actively managed by this class for now

        # Initialize next_equipo_id based on existing data
        self.next_equipo_id = 1
        if self.equipos:
            max_id = 0
            for eq_id_str in self.equipos.keys():
                try:
                    max_id = max(max_id, int(eq_id_str))
                except ValueError:
                    pass # Handle non-numeric IDs gracefully
            self.next_equipo_id = max_id + 1

    def registrar_equipo(self, nombre, marca, modelo, serial, linea):
        """Registers a new equipment, generating a unique ID."""
        if not all([nombre, marca, modelo, serial, linea]):
            return "Error: Todos los campos del equipo son obligatorios."

        id_equipo = str(self.next_equipo_id)
        while id_equipo in self.equipos:
            self.next_equipo_id += 1
            id_equipo = str(self.next_equipo_id)

        self.equipos[id_equipo] = {
            "NOMBRE": nombre,
            "MARCA": marca,
            "MODELO": modelo,
            "SERIAL": serial,
            "LINEA": linea,
            "FECHA_REGISTRO": datetime.now().strftime("%Y-%m-%d")
        }
        self.next_equipo_id += 1
        return f"Equipo registrado exitosamente! ID: {id_equipo}"

    def get_equipo(self, equipo_id):
        """Retrieves equipment data by ID."""
        return self.equipos.get(equipo_id)

    def listar_equipos(self):
        """Retrieves all equipment data and returns it as a pandas DataFrame."""
        if not self.equipos:
            # Return an empty DataFrame with expected columns if no equipment
            return pd.DataFrame(columns=["ID","NOMBRE", "MARCA", "MODELO", "SERIAL", "LINEA", "FECHA_REGISTRO"])
        df = pd.DataFrame.from_dict(self.equipos, orient='index')
        df.index.name = 'ID'
        return df

    def get_equipo_options(self):
         """Retrieves equipment options in the format 'ID - Nombre' for dropdowns."""
         return [f"{eq_id} - {data.get('NOMBRE', 'Nombre Desconocido')}" for eq_id, data in self.equipos.items()]


    def actualizar_equipo(self, equipo_id, nombre, marca, modelo, serial, linea):
        """Updates existing equipment data."""
        if equipo_id not in self.equipos:
            return "Error: El ID del equipo no existe."
        if not all([nombre, marca, modelo, serial, linea]):
            return "Error: Todos los campos del equipo son obligatorios."

        self.equipos[equipo_id]["NOMBRE"] = nombre
        self.equipos[equipo_id]["MARCA"] = marca
        self.equipos[equipo_id]["MODELO"] = modelo
        self.equipos[equipo_id]["SERIAL"] = serial
        self.equipos[equipo_id]["LINEA"] = linea
        return f"El equipo con ID {equipo_id} ha sido actualizado."

    def eliminar_equipo(self, equipo_id):
        """Deletes equipment data and its associated maintenance and calibration records."""
        if equipo_id not in self.equipos:
            return "Error: El ID del equipo no existe."

        del self.equipos[equipo_id]
        if equipo_id in self.mantenimiento:
            del self.mantenimiento[equipo_id]
        if equipo_id in self.calibracion:
            del self.calibracion[equipo_id]

        # Recalculate next_equipo_id
        if self.equipos:
            max_id = 0
            for eq_id_str in self.equipos.keys():
                try:
                    max_id = max(max_id, int(eq_id_str))
                except ValueError:
                    pass
            self.next_equipo_id = max_id + 1
        else:
            self.next_equipo_id = 1

        return f"El equipo con ID {equipo_id} y sus registros asociados han sido eliminados."

    def registrar_mantenimiento(self, equipo_id, fecha_programada_str, descripcion, tecnico):
        """Registers maintenance for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if not all([fecha_programada_str, descripcion, tecnico]):
            return "Error: Todos los campos del mantenimiento son obligatorios."
        try:
            datetime.strptime(fecha_programada_str, "%Y-%m-%d") # Validate date format
        except ValueError:
             return "Error: Formato de fecha programada inválido. Use YYYY-MM-DD."


        registro = {
            "FECHA": datetime.now().strftime("%Y-%m-%d"),
            "FECHA_PROGRAMADA": fecha_programada_str,
            "DESCRIPCION": descripcion,
            "TECNICO": tecnico
        }
        if equipo_id not in self.mantenimiento:
            self.mantenimiento[equipo_id] = []
        self.mantenimiento[id_equipo].append(registro)
        return "Mantenimiento registrado exitosamente."

    def get_mantenimiento(self, equipo_id):
        """Retrieves maintenance records for an equipment."""
        if equipo_id not in self.equipos:
            return "Error: El ID del equipo no existe."
        if equipo_id in self.mantenimiento and self.mantenimiento[equipo_id]:
            df = pd.DataFrame(self.mantenimiento[equipo_id])
            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
            return f"\nHISTORIAL DE MANTENIMIENTO EQUIPO {equipo_id} ({equipo_nombre}):\n\n{df.to_string(index=False)}\n"
        else:
            return "No hay registros de mantenimiento para este equipo."

    def get_mantenimiento_options(self, equipo_id):
        """Retrieves maintenance options for an equipment in a formatted string list."""
        if equipo_id in self.mantenimiento and self.mantenimiento[equipo_id]:
            maint_records = []
            for i, record in enumerate(self.mantenimiento[equipo_id]):
                 maint_records.append(f"{i+1}: Fecha Prog: {record.get('FECHA_PROGRAMADA', 'N/A')}, Desc: {record.get('DESCRIPCION', 'N/A')}, Tec: {record.get('TECNICO', 'N/A')}")
            return maint_records
        return []

    def eliminar_mantenimiento(self, equipo_id, record_index):
        """Deletes a specific maintenance record by index for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if equipo_id in self.mantenimiento and len(self.mantenimiento[equipo_id]) > record_index >= 0:
            del self.mantenimiento[equipo_id][record_index]
            return "Registro de mantenimiento eliminado exitosamente."
        else:
            return "Error: Registro de mantenimiento no encontrado."

    def registrar_calibracion(self, equipo_id, fecha_programada_str, descripcion, tecnico):
        """Registers calibration for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if not all([fecha_programada_str, descripcion, tecnico]):
            return "Error: Todos los campos de la calibración son obligatorios."
        try:
            datetime.strptime(fecha_programada_str, "%Y-%m-%d") # Validate date format
        except ValueError:
             return "Error: Formato de fecha programada inválido. Use YYYY-MM-DD."

        registro = {
            "FECHA": datetime.now().strftime("%Y-%m-%d"),
            "FECHA_PROGRAMADA": fecha_programada_str,
            "DESCRIPCION": descripcion,
            "TECNICO": tecnico
        }
        if equipo_id not in self.calibracion:
            self.calibracion[equipo_id] = []
        self.calibracion[id_equipo].append(registro)
        return "Calibración registrada exitosamente."

    def get_calibracion(self, equipo_id):
        """Retrieves calibration records for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if equipo_id in self.calibracion and self.calibracion[equipo_id]:
            df = pd.DataFrame(self.calibracion[equipo_id])
            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
            return f"\nHISTORIAL DE CALIBRACIÓN EQUIPO {equipo_id} ({equipo_nombre}):\n\n{df.to_string(index=False)}\n"
        else:
            return "No hay registros de calibración para este equipo."

    def get_calibracion_options(self, equipo_id):
        """Retrieves calibration options for an equipment in a formatted string list."""
        if equipo_id in self.calibracion and self.calibracion[equipo_id]:
            calib_records = []
            for i, record in enumerate(self.calibracion[equipo_id]):
                 calib_records.append(f"{i+1}: Fecha Prog: {record.get('FECHA_PROGRAMADA', 'N/A')}, Desc: {record.get('DESCRIPCION', 'N/A')}, Tec: {record.get('TECNICO', 'N/A')}")
            return calib_records
        return []

    def eliminar_calibracion(self, equipo_id, record_index):
        """Deletes a specific calibration record by index for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if equipo_id in self.calibracion and len(self.calibracion[equipo_id]) > record_index >= 0:
            del self.calibracion[equipo_id][record_index]
            return "Registro de calibración eliminado exitosamente."
        else:
            return "Error: Registro de calibración no encontrado."

    def check_alerts(self):
        """Checks for upcoming maintenance and calibration alerts."""
        today = datetime.now().date()
        upcoming_alerts = []

        # Check Maintenance Alerts
        for equipo_id, mantenimientos in self.mantenimiento.items():
            for mantenimiento in mantenimientos:
                if "FECHA_PROGRAMADA" in mantenimiento:
                    try:
                        scheduled_date = datetime.strptime(mantenimiento["FECHA_PROGRAMADA"], "%Y-%m-%d").date()
                        if scheduled_date >= today:
                            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
                            days_until = (scheduled_date - today).days
                            alert_message = f"Próximo Mantenimiento: Equipo {equipo_id} ({equipo_nombre}) : Fecha Programada: {mantenimiento['FECHA_PROGRAMADA']}"
                            if days_until == 0:
                                alert_message += " (HOY)"
                            elif days_until > 0:
                                alert_message += f" (en {days_until} días)"
                            upcoming_alerts.append(alert_message)
                    except ValueError:
                        print(f"Warning: Could not parse date from maintenance record: {mantenimiento}")

        # Check Calibration Alerts
        for equipo_id, calibraciones in self.calibracion.items():
            for calibracion in calibraciones:
                if "FECHA_PROGRAMADA" in calibracion:
                    try:
                        scheduled_date = datetime.strptime(calibracion["FECHA_PROGRAMADA"], "%Y-%m-%d").date()
                        if scheduled_date >= today:
                            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
                            days_until = (scheduled_date - today).days
                            alert_message = f"Próxima Calibración: Equipo {equipo_id} ({equipo_nombre}) : Fecha Programada: {calibracion['FECHA_PROGRAMADA']}"
                            if days_until == 0:
                                alert_message += " (HOY)"
                            elif days_until > 0:
                                alert_message += f" (en {days_until} días)"
                            upcoming_alerts.append(alert_message)
                    except ValueError:
                        print(f"Warning: Could not parse date from calibration record: {calibracion}")

        if upcoming_alerts:
            return "Alertas de Próximos Eventos:\n\n" + "\n".join(upcoming_alerts)
        else:
            return "No hay alertas de próximos eventos."

# --- Load initial data and create GestorDatos instance ---
equipos_data, mantenimiento_data, calibracion_data, alertas_data = cargar_datos()
gestor = GestorDatos(equipos_data, mantenimiento_data, calibracion_data, alertas_data)

# --- Define Gradio Components (Dropdowns used across multiple tabs) ---
update_equipo_dropdown = gr.Dropdown(label="ID - Nombre", choices=gestor.get_equipo_options(), interactive=True)
delete_equipo_dropdown = gr.Dropdown(label="ID - Nombre", choices=gestor.get_equipo_options(), interactive=True)
maint_equipo_dropdown_reg = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
maint_equipo_dropdown_view = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
maint_equipo_dropdown_del = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
calib_equipo_dropdown_reg = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
calib_equipo_dropdown_view = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
calib_equipo_dropdown_del = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)

all_equipo_dropdowns = [
    update_equipo_dropdown,
    delete_equipo_dropdown,
    maint_equipo_dropdown_reg,
    maint_equipo_dropdown_view,
    maint_equipo_dropdown_del,
    calib_equipo_dropdown_reg,
    calib_equipo_dropdown_view,
    calib_equipo_dropdown_del
]

# --- Helper function to update all equipment dropdowns ---
def update_all_equipo_dropdowns():
    new_options = gestor.get_equipo_options()
    # Return updates for each dropdown component
    return [gr.Dropdown(choices=new_options, value=None)] * len(all_equipo_dropdowns)

# --- Gradio Interface Definition ---
with gr.Blocks(title=".::-REGISTRO HOJA DE VIDA EQUIPOS CDA-::.") as app:
    gr.Label(".::-REGISTRO HOJA DE VIDA EQUIPOS CDA-::. VERSION: ALFA 1.0.8")

    with gr.Tabs():
        # Equipos Tab
        with gr.TabItem("GESTION DE EQUIPOS"):
            with gr.Accordion("Registrar Equipo"):
                reg_nombre = gr.Textbox(label="Nombre")
                reg_marca = gr.Textbox(label="Marca")
                reg_modelo = gr.Textbox(label="Modelo")
                reg_serial = gr.Textbox(label="Serial")
                reg_linea = gr.Dropdown(label="Linea", choices=["Pista Livianos", "Pista Motocicletas", "Pista Pesados"])
                btn_registrar = gr.Button("Registrar Equipo")
                reg_output = gr.Textbox(label="Estado", interactive=False)

                def register_equipo(nombre, marca, modelo, serial, linea):
                    result = gestor.registrar_equipo(nombre, marca, modelo, serial, linea)
                    dropdown_updates = update_all_equipo_dropdowns()
                    return (result,) + tuple(dropdown_updates)

                btn_registrar.click(
                    register_equipo,
                    inputs=[reg_nombre, reg_marca, reg_modelo, reg_serial, reg_linea],
                    outputs=[reg_output] + all_equipo_dropdowns
                )

            with gr.Accordion("Ver Equipos Registrados"):
                btn_ver_equipos = gr.Button("Ver Equipos Registrados")
                ver_equipos_output = gr.Dataframe(label="Lista de Equipos", interactive=False)

                def display_equipos():
                    return gestor.listar_equipos()

                btn_ver_equipos.click(display_equipos, outputs=ver_equipos_output)

            with gr.Accordion("Actualizar Equipo"):
                update_nombre = gr.Textbox(label="Nombre")
                update_marca = gr.Textbox(label="Marca")
                update_modelo = gr.Textbox(label="Modelo")
                update_serial = gr.Textbox(label="Serial")
                update_linea = gr.Dropdown(label="Linea", choices=["Pista Livianos", "Pista Motocicletas", "Pista Pesados"])
                btn_actualizar = gr.Button("Guardar Actualización")
                update_output = gr.Textbox(label="Estado", interactive=False)

                def load_equipo_for_update(selected_option):
                    if not selected_option:
                        return "", "", "", "", ""
                    equipo_id = selected_option.split(' - ')[0]
                    equipo_data = gestor.get_equipo(equipo_id)
                    if equipo_data:
                        return equipo_data.get('NOMBRE', ''), equipo_data.get('MARCA', ''), equipo_data.get('MODELO', ''), equipo_data.get('SERIAL', ''), equipo_data.get('LINEA', '')
                    return "", "", "", "", ""

                update_equipo_dropdown.change(
                    load_equipo_for_update,
                    inputs=update_equipo_dropdown,
                    outputs=[update_nombre, update_marca, update_modelo, update_serial, update_linea]
                )

                def update_equipo(selected_option, nombre, marca, modelo, serial, linea):
                    if not selected_option:
                        return "Error: Seleccione un equipo para actualizar.", *update_all_equipo_dropdowns()
                    equipo_id = selected_option.split(' - ')[0]
                    result = gestor.actualizar_equipo(equipo_id, nombre, marca, modelo, serial, linea)
                    dropdown_updates = update_all_equipo_dropdowns()
                    return (result,) + tuple(dropdown_updates)

                btn_actualizar.click(
                    update_equipo,
                    inputs=[update_equipo_dropdown, update_nombre, update_marca, update_modelo, update_serial, update_linea],
                    outputs=[update_output] + all_equipo_dropdowns
                )

            with gr.Accordion("Eliminar Equipo"):
                btn_eliminar_equipo = gr.Button("Eliminar Equipo")
                delete_equipo_output = gr.Textbox(label="Estado", interactive=False)

                def delete_equipo(selected_option):
                     if not selected_option:
                         return "Error: Seleccione un equipo para eliminar.", *update_all_equipo_dropdowns()
                     equipo_id = selected_option.split(' - ')[0]
                     result = gestor.eliminar_equipo(equipo_id)
                     dropdown_updates = update_all_equipo_dropdowns()
                     return (result,) + tuple(dropdown_updates)

                btn_eliminar_equipo.click(
                     delete_equipo,
                     inputs=delete_equipo_dropdown,
                     outputs=[delete_equipo_output] + all_equipo_dropdowns
                )

        # Mantenimientos Tab
        with gr.TabItem("GESTION DE MANTENIMIENTOS"):
             with gr.Accordion("Programar Mantenimiento"):
                 maint_fecha = gr.Textbox(label="Fecha Programada (YYYY-MM-DD)", placeholder="YYYY-MM-DD")
                 maint_desc = gr.Textbox(label="Descripción")
                 maint_tec = gr.Textbox(label="Técnico responsable")
                 btn_registrar_maint = gr.Button("Programar Mantenimiento")
                 maint_reg_output = gr.Textbox(label="Estado", interactive=False)

                 def register_mantenimiento(selected_option, fecha_str, desc, tec):
                     if not selected_option:
                          return "Error: Seleccione un equipo."
                     if not fecha_str:
                          return "Error: Ingrese una fecha."
                     equipo_id = selected_option.split(' - ')[0]
                     result = gestor.registrar_mantenimiento(equipo_id, fecha_str, desc, tec)
                     return result

                 btn_registrar_maint.click(
                      register_mantenimiento,
                      inputs=[maint_equipo_dropdown_reg, maint_fecha, maint_desc, maint_tec],
                      outputs=maint_reg_output
                 )

             with gr.Accordion("Historial Mantenimiento"):
                  btn_ver_maint = gr.Button("Ver Historial de Mantenimiento")
                  ver_maint_output = gr.TextArea(label="Historial de Mantenimiento", interactive=False)

                  def display_mantenimiento(selected_option):
                       if not selected_option:
                            return "Error: Seleccione un equipo."
                       equipo_id = selected_option.split(' - ')[0]
                       return gestor.get_mantenimiento(equipo_id)

                  btn_ver_maint.click(
                       display_mantenimiento,
                       inputs=maint_equipo_dropdown_view,
                       outputs=ver_maint_output
                  )

             with gr.Accordion("Eliminar Mantenimiento"):
                 maint_record_dropdown_del = gr.Dropdown(label="Seleccione el registro de mantenimiento:", choices=[], interactive=True)
                 btn_eliminar_maint = gr.Button("Eliminar Mantenimiento Seleccionado")
                 maint_del_output = gr.Textbox(label="Estado", interactive=False)

                 def load_maint_records_for_delete(selected_equipo_option):
                      if not selected_equipo_option:
                          return gr.Dropdown(choices=[], value=None, interactive=True)
                      equipo_id = selected_equipo_option.split(' - ')[0]
                      options = gestor.get_mantenimiento_options(equipo_id)
                      return gr.Dropdown(choices=options, value=options[0] if options else None, interactive=True)

                 maint_equipo_dropdown_del.change(
                      load_maint_records_for_delete,
                      inputs=maint_equipo_dropdown_del,
                      outputs=maint_record_dropdown_del
                 )

                 def delete_mantenimiento(selected_equipo_option, selected_maint_option):
                      if not selected_equipo_option:
                          return "Error: Seleccione un equipo.", gr.Dropdown(choices=[], value=None, interactive=True)
                      if not selected_maint_option:
                           return "Error: Seleccione un registro de mantenimiento.", gr.Dropdown(choices=[], value=None, interactive=True)

                      equipo_id = selected_equipo_option.split(' - ')[0]
                      try:
                          record_index_str = selected_maint_option.split(':')[0]
                          record_index = int(record_index_str) - 1
                      except (ValueError, IndexError):
                           return "Error: Formato de registro de mantenimiento inválido.", gr.Dropdown(choices=[], value=None, interactive=True)

                      result = gestor.eliminar_mantenimiento(equipo_id, record_index)
                      new_options = gestor.get_mantenimiento_options(equipo_id)
                      return result, gr.Dropdown(choices=new_options, value=new_options[0] if new_options else None, interactive=True)

                 btn_eliminar_maint.click(
                      delete_mantenimiento,
                      inputs=[maint_equipo_dropdown_del, maint_record_dropdown_del],
                      outputs=[maint_del_output, maint_record_dropdown_del]
                 )

        # Calibraciones Tab
        with gr.TabItem("GESTION DE CALIBRACIONES"):
             with gr.Accordion("Programar Calibración"):
                 calib_fecha = gr.Textbox(label="Fecha Programada (YYYY-MM-DD)", placeholder="YYYY-MM-DD")
                 calib_desc = gr.Textbox(label="Descripción")
                 calib_tec = gr.Textbox(label="Técnico responsable")
                 btn_registrar_calib = gr.Button("Programar Calibración")
                 calib_reg_output = gr.Textbox(label="Estado", interactive=False)

                 def register_calibracion(selected_option, fecha_str, desc, tec):
                      if not selected_option:
                          return "Error: Seleccione un equipo."
                      if not fecha_str:
                           return "Error: Ingrese una fecha."
                      equipo_id = selected_option.split(' - ')[0]
                      result = gestor.registrar_calibracion(equipo_id, fecha_str, desc, tec)
                      return result

                 btn_registrar_calib.click(
                      register_calibracion,
                      inputs=[calib_equipo_dropdown_reg, calib_fecha, calib_desc, calib_tec],
                      outputs=calib_reg_output
                 )

             with gr.Accordion("Historial De Calibración"):
                  btn_ver_calib = gr.Button("Ver Historial De Calibración")
                  ver_calib_output = gr.TextArea(label="Historial De Calibración", interactive=False)

                  def display_calibracion(selected_option):
                       if not selected_option:
                            return "Error: Seleccione un equipo."
                       equipo_id = selected_option.split(' - ')[0]
                       return gestor.get_calibracion(equipo_id)

                  btn_ver_calib.click(
                       display_calibracion,
                       inputs=calib_equipo_dropdown_view,
                       outputs=ver_calib_output
                  )

             with gr.Accordion("Eliminar Calibración"):
                  calib_record_dropdown_del = gr.Dropdown(label="Seleccione el registro de calibración:", choices=[], interactive=True)
                  btn_eliminar_calib = gr.Button("Eliminar Calibración Programada")
                  calib_del_output = gr.Textbox(label="Estado", interactive=False)

                  def load_calib_records_for_delete(selected_equipo_option):
                      if not selected_equipo_option:
                          return gr.Dropdown(choices=[], value=None, interactive=True)
                      equipo_id = selected_equipo_option.split(' - ')[0]
                      options = gestor.get_calibracion_options(equipo_id)
                      return gr.Dropdown(choices=options, value=options[0] if options else None, interactive=True)

                  calib_equipo_dropdown_del.change(
                       load_calib_records_for_delete,
                       inputs=calib_equipo_dropdown_del,
                       outputs=calib_record_dropdown_del
                  )

                  def delete_calibracion(selected_equipo_option, selected_calib_option):
                       if not selected_equipo_option:
                           return "Error: Seleccione un equipo.", gr.Dropdown(choices=[], value=None, interactive=True)
                       if not selected_calib_option:
                            return "Error: Seleccione un registro de calibración.", gr.Dropdown(choices=[], value=None, interactive=True)

                       equipo_id = selected_equipo_option.split(' - ')[0]
                       try:
                            record_index_str = selected_calib_option.split(':')[0]
                            record_index = int(record_index_str) - 1
                       except (ValueError, IndexError):
                            return "Error: Formato de registro de calibración inválido.", gr.Dropdown(choices=[], value=None, interactive=True)

                       result = gestor.eliminar_calibracion(equipo_id, record_index)
                       new_options = gestor.get_calibracion_options(equipo_id)
                       return result, gr.Dropdown(choices=new_options, value=new_options[0] if new_options else None, interactive=True)

                  btn_eliminar_calib.click(
                       delete_calibracion,
                       inputs=[calib_equipo_dropdown_del, calib_record_dropdown_del],
                       outputs=[calib_del_output, calib_record_dropdown_del]
                  )

        # Alerts Section (separate tab)
        with gr.TabItem("Alertas"):
             btn_check_alerts = gr.Button("Ver Alertas")
             alerts_output = gr.TextArea(label="Alertas de Próximos Eventos", interactive=False)

             def check_alerts():
                 return gestor.check_alerts()

             btn_check_alerts.click(check_alerts, outputs=alerts_output)

    # Save Data Button (outside tabs for easy access)
    with gr.Row():
        btn_save_data = gr.Button("Guardar Datos")
        save_output = gr.Textbox(label="Estado de Guardado", interactive=False)

    def save_data():
        guardar_datos(gestor.equipos, gestor.mantenimiento, gestor.calibracion, gestor.alertas)
        return "Datos guardados exitosamente!"

    btn_save_data.click(save_data, outputs=save_output)

    # Register the save_data function to be called when the Gradio app is closed
    # Note: Auto-saving on close using browser events is not fully reliable.
    # A dedicated save button is the recommended approach.
    app.load(
        None,
        None,
        js="""
        function() {
            const saveButton = document.querySelector('button[data-testid="save_data"]');
            if (saveButton) {
                window.addEventListener('beforeunload', (event) => {
                    saveButton.click();
                });
            } else {
                console.log("Save button not found. Auto-save on close will not work.");
            }
        }
        """
    )

# --- Main Execution Block ---
if __name__ == "__main__":
    app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4f59b1d7ea14b1419c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [3]:
import json
import os
from datetime import datetime, timedelta
import pandas as pd
import gradio as gr

# --- Data Loading and Saving Functions ---
EQUIPOS_FILE = '/content/equipos.json'
MANTENIMIENTO_FILE = '/content/mantenimiento.json'
CALIBRACIONES_FILE = '/content/calibraciones.json'
ALERTS_FILE = 'alertas.json'

def cargar_datos():
    """Loads data from JSON files into dictionaries."""
    def load_json(filename):
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                try:
                    return json.load(f)
                except json.JSONDecodeError:
                    print(f"Warning: Could not decode JSON from {filename}. Returning empty dictionary.")
                    return {}
        else:
            return {}

    equipos = load_json(EQUIPOS_FILE)
    # Ensure 'NOMBRE' key exists for all equipment entries for robustness
    for eq_id in equipos:
        if 'NOMBRE' not in equipos[eq_id]:
            equipos[eq_id]['NOMBRE'] = 'Nombre Desconocido' # Provide a default name

    mantenimiento = load_json(MANTENIMIENTO_FILE)
    calibracion = load_json(CALIBRACIONES_FILE)
    alertas = load_json(ALERTS_FILE) # Load alerts data, though not used for saving/loading alerts in Tkinter code

    return equipos, mantenimiento, calibracion, alertas

def guardar_datos(equipos, mantenimiento, calibracion, alertas):
    """Saves data from dictionaries into JSON files."""
    def save_json(data, filename):
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)

    save_json(equipos, EQUIPOS_FILE)
    save_json(mantenimiento, MANTENIMIENTO_FILE)
    save_json(calibracion, CALIBRACIONES_FILE)
    save_json(alertas, ALERTS_FILE) # Save alerts data for consistency, though not used in Tkinter's saving logic

# --- GestorDatos Class (Core Logic) ---
class GestorDatos:
    """Encapsulates the core logic for managing equipment, maintenance, and calibration data."""
    def __init__(self, equipos, mantenimiento, calibracion, alertas):
        self.equipos = equipos
        self.mantenimiento = mantenimiento
        self.calibracion = calibracion
        self.alertas = alertas # alerts data is loaded but not actively managed by this class for now

        # Initialize next_equipo_id based on existing data
        self.next_equipo_id = 1
        if self.equipos:
            max_id = 0
            for eq_id_str in self.equipos.keys():
                try:
                    max_id = max(max_id, int(eq_id_str))
                except ValueError:
                    pass # Handle non-numeric IDs gracefully
            self.next_equipo_id = max_id + 1

    def registrar_equipo(self, nombre, marca, modelo, serial, linea):
        """Registers a new equipment, generating a unique ID."""
        if not all([nombre, marca, modelo, serial, linea]):
            return "Error: Todos los campos del equipo son obligatorios."

        id_equipo = str(self.next_equipo_id)
        while id_equipo in self.equipos:
            self.next_equipo_id += 1
            id_equipo = str(self.next_equipo_id)

        self.equipos[id_equipo] = {
            "NOMBRE": nombre,
            "MARCA": marca,
            "MODELO": modelo,
            "SERIAL": serial,
            "LINEA": linea,
            "FECHA_REGISTRO": datetime.now().strftime("%Y-%m-%d")
        }
        self.next_equipo_id += 1
        return f"Equipo registrado exitosamente! ID: {id_equipo}"

    def get_equipo(self, equipo_id):
        """Retrieves equipment data by ID."""
        return self.equipos.get(equipo_id)

    def listar_equipos(self):
        """Retrieves all equipment data and returns it as a pandas DataFrame."""
        if not self.equipos:
            # Return an empty DataFrame with expected columns if no equipment
            return pd.DataFrame(columns=["ID","NOMBRE", "MARCA", "MODELO", "SERIAL", "LINEA", "FECHA_REGISTRO"])
        df = pd.DataFrame.from_dict(self.equipos, orient='index')
        df.index.name = 'ID'
        return df

    def get_equipo_options(self):
         """Retrieves equipment options in the format 'ID - Nombre' for dropdowns."""
         return [f"{eq_id} - {data.get('NOMBRE', 'Nombre Desconocido')}" for eq_id, data in self.equipos.items()]


    def actualizar_equipo(self, equipo_id, nombre, marca, modelo, serial, linea):
        """Updates existing equipment data."""
        if equipo_id not in self.equipos:
            return "Error: El ID del equipo no existe."
        if not all([nombre, marca, modelo, serial, linea]):
            return "Error: Todos los campos del equipo son obligatorios."

        self.equipos[equipo_id]["NOMBRE"] = nombre
        self.equipos[equipo_id]["MARCA"] = marca
        self.equipos[equipo_id]["MODELO"] = modelo
        self.equipos[equipo_id]["SERIAL"] = serial
        self.equipos[equipo_id]["LINEA"] = linea
        return f"El equipo con ID {equipo_id} ha sido actualizado."

    def eliminar_equipo(self, equipo_id):
        """Deletes equipment data and its associated maintenance and calibration records."""
        if equipo_id not in self.equipos:
            return "Error: El ID del equipo no existe."

        del self.equipos[equipo_id]
        if equipo_id in self.mantenimiento:
            del self.mantenimiento[equipo_id]
        if equipo_id in self.calibracion:
            del self.calibracion[equipo_id]

        # Recalculate next_equipo_id
        if self.equipos:
            max_id = 0
            for eq_id_str in self.equipos.keys():
                try:
                    max_id = max(max_id, int(eq_id_str))
                except ValueError:
                    pass
            self.next_equipo_id = max_id + 1
        else:
            self.next_equipo_id = 1

        return f"El equipo con ID {equipo_id} y sus registros asociados han sido eliminados."

    def registrar_mantenimiento(self, equipo_id, fecha_programada_str, descripcion, tecnico):
        """Registers maintenance for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if not all([fecha_programada_str, descripcion, tecnico]):
            return "Error: Todos los campos del mantenimiento son obligatorios."
        try:
            datetime.strptime(fecha_programada_str, "%Y-%m-%d") # Validate date format
        except ValueError:
             return "Error: Formato de fecha programada inválido. Use YYYY-MM-DD."


        registro = {
            "FECHA": datetime.now().strftime("%Y-%m-%d"),
            "FECHA_PROGRAMADA": fecha_programada_str,
            "DESCRIPCION": descripcion,
            "TECNICO": tecnico
        }
        if equipo_id not in self.mantenimiento:
            self.mantenimiento[equipo_id] = []
        self.mantenimiento[equipo_id].append(registro)
        return "Mantenimiento registrado exitosamente."

    def get_mantenimiento(self, equipo_id):
        """Retrieves maintenance records for an equipment."""
        if equipo_id not in self.equipos:
            return "Error: El ID del equipo no existe."
        if equipo_id in self.mantenimiento and self.mantenimiento[equipo_id]:
            df = pd.DataFrame(self.mantenimiento[equipo_id])
            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
            return f"\nHISTORIAL DE MANTENIMIENTO EQUIPO {equipo_id} ({equipo_nombre}):\n\n{df.to_string(index=False)}\n"
        else:
            return "No hay registros de mantenimiento para este equipo."

    def get_mantenimiento_options(self, equipo_id):
        """Retrieves maintenance options for an equipment in a formatted string list."""
        if equipo_id in self.mantenimiento and self.mantenimiento[equipo_id]:
            maint_records = []
            for i, record in enumerate(self.mantenimiento[equipo_id]):
                 maint_records.append(f"{i+1}: Fecha Prog: {record.get('FECHA_PROGRAMADA', 'N/A')}, Desc: {record.get('DESCRIPCION', 'N/A')}, Tec: {record.get('TECNICO', 'N/A')}")
            return maint_records
        return []

    def eliminar_mantenimiento(self, equipo_id, record_index):
        """Deletes a specific maintenance record by index for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if equipo_id in self.mantenimiento and len(self.mantenimiento[equipo_id]) > record_index >= 0:
            del self.mantenimiento[equipo_id][record_index]
            return "Registro de mantenimiento eliminado exitosamente."
        else:
            return "Error: Registro de mantenimiento no encontrado."

    def registrar_calibracion(self, equipo_id, fecha_programada_str, descripcion, tecnico):
        """Registers calibration for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if not all([fecha_programada_str, descripcion, tecnico]):
            return "Error: Todos los campos de la calibración son obligatorios."
        try:
            datetime.strptime(fecha_programada_str, "%Y-%m-%d") # Validate date format
        except ValueError:
             return "Error: Formato de fecha programada inválido. Use YYYY-MM-DD."

        registro = {
            "FECHA": datetime.now().strftime("%Y-%m-%d"),
            "FECHA_PROGRAMADA": fecha_programada_str,
            "DESCRIPCION": descripcion,
            "TECNICO": tecnico
        }
        if equipo_id not in self.calibracion:
            self.calibracion[equipo_id] = []
        self.calibracion[equipo_id].append(registro)
        return "Calibración registrada exitosamente."

    def get_calibracion(self, equipo_id):
        """Retrieves calibration records for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if equipo_id in self.calibracion and self.calibracion[equipo_id]:
            df = pd.DataFrame(self.calibracion[equipo_id])
            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
            return f"\nHISTORIAL DE CALIBRACIÓN EQUIPO {equipo_id} ({equipo_nombre}):\n\n{df.to_string(index=False)}\n"
        else:
            return "No hay registros de calibración para este equipo."

    def get_calibracion_options(self, equipo_id):
        """Retrieves calibration options for an equipment in a formatted string list."""
        if equipo_id in self.calibracion and self.calibracion[equipo_id]:
            calib_records = []
            for i, record in enumerate(self.calibracion[equipo_id]):
                 calib_records.append(f"{i+1}: Fecha Prog: {record.get('FECHA_PROGRAMADA', 'N/A')}, Desc: {record.get('DESCRIPCION', 'N/A')}, Tec: {record.get('TECNICO', 'N/A')}")
            return calib_records
        return []

    def eliminar_calibracion(self, equipo_id, record_index):
        """Deletes a specific calibration record by index for an equipment."""
        if equipo_id not in self.equipos:
             return "Error: El ID del equipo no existe."
        if equipo_id in self.calibracion and len(self.calibracion[equipo_id]) > record_index >= 0:
            del self.calibracion[equipo_id][record_index]
            return "Registro de calibración eliminado exitosamente."
        else:
            return "Error: Registro de calibración no encontrado."

    def check_alerts(self):
        """Checks for upcoming maintenance and calibration alerts."""
        today = datetime.now().date()
        upcoming_alerts = []

        # Check Maintenance Alerts
        for equipo_id, mantenimientos in self.mantenimiento.items():
            for mantenimiento in mantenimientos:
                if "FECHA_PROGRAMADA" in mantenimiento:
                    try:
                        scheduled_date = datetime.strptime(mantenimiento["FECHA_PROGRAMADA"], "%Y-%m-%d").date()
                        if scheduled_date >= today:
                            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
                            days_until = (scheduled_date - today).days
                            alert_message = f"Próximo Mantenimiento: Equipo {equipo_id} ({equipo_nombre}) : Fecha Programada: {mantenimiento['FECHA_PROGRAMADA']}"
                            if days_until == 0:
                                alert_message += " (HOY)"
                            elif days_until > 0:
                                alert_message += f" (en {days_until} días)"
                            upcoming_alerts.append(alert_message)
                    except ValueError:
                        print(f"Warning: Could not parse date from maintenance record: {mantenimiento}")

        # Check Calibration Alerts
        for equipo_id, calibraciones in self.calibracion.items():
            for calibracion in calibraciones:
                if "FECHA_PROGRAMADA" in calibracion:
                    try:
                        scheduled_date = datetime.strptime(calibracion["FECHA_PROGRAMADA"], "%Y-%m-%d").date()
                        if scheduled_date >= today:
                            equipo_nombre = self.equipos.get(equipo_id, {}).get('NOMBRE', 'Desconocido')
                            days_until = (scheduled_date - today).days
                            alert_message = f"Próxima Calibración: Equipo {equipo_id} ({equipo_nombre}) : Fecha Programada: {calibracion['FECHA_PROGRAMADA']}"
                            if days_until == 0:
                                alert_message += " (HOY)"
                            elif days_until > 0:
                                alert_message += f" (en {days_until} días)"
                            upcoming_alerts.append(alert_message)
                    except ValueError:
                        print(f"Warning: Could not parse date from calibration record: {calibracion}")

        if upcoming_alerts:
            return "Alertas de Próximos Eventos:\n\n" + "\n".join(upcoming_alerts)
        else:
            return "No hay alertas de próximos eventos."

# --- Load initial data and create GestorDatos instance ---
equipos_data, mantenimiento_data, calibracion_data, alertas_data = cargar_datos()
gestor = GestorDatos(equipos_data, mantenimiento_data, calibracion_data, alertas_data)

# --- Define Gradio Components (Dropdowns used across multiple tabs) ---
update_equipo_dropdown = gr.Dropdown(label="ID - Nombre", choices=gestor.get_equipo_options(), interactive=True)
delete_equipo_dropdown = gr.Dropdown(label="ID - Nombre", choices=gestor.get_equipo_options(), interactive=True)
maint_equipo_dropdown_reg = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
maint_equipo_dropdown_view = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
maint_equipo_dropdown_del = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
calib_equipo_dropdown_reg = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
calib_equipo_dropdown_view = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)
calib_equipo_dropdown_del = gr.Dropdown(label="Seleccione el equipo:", choices=gestor.get_equipo_options(), interactive=True)

all_equipo_dropdowns = [
    update_equipo_dropdown,
    delete_equipo_dropdown,
    maint_equipo_dropdown_reg,
    maint_equipo_dropdown_view,
    maint_equipo_dropdown_del,
    calib_equipo_dropdown_reg,
    calib_equipo_dropdown_view,
    calib_equipo_dropdown_del
]

# --- Helper function to update all equipment dropdowns ---
def update_all_equipo_dropdowns():
    new_options = gestor.get_equipo_options()
    # Return updates for each dropdown component
    return [gr.Dropdown(choices=new_options, value=None)] * len(all_equipo_dropdowns)

# --- Gradio Interface Definition ---
with gr.Blocks(title=".::-REGISTRO HOJA DE VIDA EQUIPOS CDA-::.") as app:
    gr.Label(".::-REGISTRO HOJA DE VIDA EQUIPOS CDA-::. VERSION: ALFA 1.0.8")

    with gr.Tabs():
        # Equipos Tab
        with gr.TabItem("GESTION DE EQUIPOS"):
            with gr.Accordion("Registrar Equipo"):
                reg_nombre = gr.Textbox(label="Nombre")
                reg_marca = gr.Textbox(label="Marca")
                reg_modelo = gr.Textbox(label="Modelo")
                reg_serial = gr.Textbox(label="Serial")
                reg_linea = gr.Dropdown(label="Linea", choices=["Pista Livianos", "Pista Motocicletas", "Pista Pesados"])
                btn_registrar = gr.Button("Registrar Equipo")
                reg_output = gr.Textbox(label="Estado", interactive=False)

                def register_equipo(nombre, marca, modelo, serial, linea):
                    result = gestor.registrar_equipo(nombre, marca, modelo, serial, linea)
                    dropdown_updates = update_all_equipo_dropdowns()
                    return (result,) + tuple(dropdown_updates)

                btn_registrar.click(
                    register_equipo,
                    inputs=[reg_nombre, reg_marca, reg_modelo, reg_serial, reg_linea],
                    outputs=[reg_output] + all_equipo_dropdowns
                )

            with gr.Accordion("Ver Equipos Registrados"):
                btn_ver_equipos = gr.Button("Ver Equipos Registrados")
                ver_equipos_output = gr.Dataframe(label="Lista de Equipos", interactive=False)

                def display_equipos():
                    return gestor.listar_equipos()

                btn_ver_equipos.click(display_equipos, outputs=ver_equipos_output)

            with gr.Accordion("Actualizar Equipo"):
                # Ensure the update_equipo_dropdown is within this accordion
                gr.Markdown("Seleccione el equipo a actualizar:")
                update_equipo_dropdown.render()
                update_nombre = gr.Textbox(label="Nombre")
                update_marca = gr.Textbox(label="Marca")
                update_modelo = gr.Textbox(label="Modelo")
                update_serial = gr.Textbox(label="Serial")
                update_linea = gr.Dropdown(label="Linea", choices=["Pista Livianos", "Pista Motocicletas", "Pista Pesados"])
                btn_actualizar = gr.Button("Guardar Actualización")
                update_output = gr.Textbox(label="Estado", interactive=False)

                def load_equipo_for_update(selected_option):
                    if not selected_option:
                        return "", "", "", "", ""
                    equipo_id = selected_option.split(' - ')[0]
                    equipo_data = gestor.get_equipo(equipo_id)
                    if equipo_data:
                        return equipo_data.get('NOMBRE', ''), equipo_data.get('MARCA', ''), equipo_data.get('MODELO', ''), equipo_data.get('SERIAL', ''), equipo_data.get('LINEA', '')
                    return "", "", "", "", ""

                update_equipo_dropdown.change(
                    load_equipo_for_update,
                    inputs=update_equipo_dropdown,
                    outputs=[update_nombre, update_marca, update_modelo, update_serial, update_linea]
                )

                def update_equipo(selected_option, nombre, marca, modelo, serial, linea):
                    if not selected_option:
                        return "Error: Seleccione un equipo para actualizar.", *update_all_equipo_dropdowns()
                    equipo_id = selected_option.split(' - ')[0]
                    result = gestor.actualizar_equipo(equipo_id, nombre, marca, modelo, serial, linea)
                    dropdown_updates = update_all_equipo_dropdowns()
                    return (result,) + tuple(dropdown_updates)

                btn_actualizar.click(
                    update_equipo,
                    inputs=[update_equipo_dropdown, update_nombre, update_marca, update_modelo, update_serial, update_linea],
                    outputs=[update_output] + all_equipo_dropdowns
                )

            with gr.Accordion("Eliminar Equipo"):
                # Ensure the delete_equipo_dropdown is within this accordion
                gr.Markdown("Seleccione el equipo a eliminar:")
                delete_equipo_dropdown.render()
                btn_eliminar_equipo = gr.Button("Eliminar Equipo")
                delete_equipo_output = gr.Textbox(label="Estado", interactive=False)

                def delete_equipo(selected_option):
                     if not selected_option:
                         return "Error: Seleccione un equipo para eliminar.", *update_all_equipo_dropdowns()
                     equipo_id = selected_option.split(' - ')[0]
                     result = gestor.eliminar_equipo(equipo_id)
                     dropdown_updates = update_all_equipo_dropdowns()
                     return (result,) + tuple(dropdown_updates)

                btn_eliminar_equipo.click(
                     delete_equipo,
                     inputs=delete_equipo_dropdown,
                     outputs=[delete_equipo_output] + all_equipo_dropdowns
                )

        # Mantenimientos Tab
        with gr.TabItem("GESTION DE MANTENIMIENTOS"):
             with gr.Accordion("Programar Mantenimiento"):
                 # Ensure the maint_equipo_dropdown_reg is within this accordion
                 gr.Markdown("Seleccione el equipo para programar mantenimiento:")
                 maint_equipo_dropdown_reg.render()
                 maint_fecha = gr.Textbox(label="Fecha Programada (YYYY-MM-DD)", placeholder="YYYY-MM-DD")
                 maint_desc = gr.Textbox(label="Descripción")
                 maint_tec = gr.Textbox(label="Técnico responsable")
                 btn_registrar_maint = gr.Button("Programar Mantenimiento")
                 maint_reg_output = gr.Textbox(label="Estado", interactive=False)

                 def register_mantenimiento(selected_option, fecha_str, desc, tec):
                     if not selected_option:
                          return "Error: Seleccione un equipo."
                     if not fecha_str:
                          return "Error: Ingrese una fecha."
                     equipo_id = selected_option.split(' - ')[0]
                     result = gestor.registrar_mantenimiento(equipo_id, fecha_str, desc, tec)
                     return result

                 btn_registrar_maint.click(
                      register_mantenimiento,
                      inputs=[maint_equipo_dropdown_reg, maint_fecha, maint_desc, maint_tec],
                      outputs=maint_reg_output
                 )

             with gr.Accordion("Historial Mantenimiento"):
                  # Use maint_equipo_dropdown_view here
                  gr.Markdown("Seleccione el equipo para ver el historial de mantenimiento:")
                  maint_equipo_dropdown_view.render()
                  ver_maint_output = gr.TextArea(label="Historial de Mantenimiento", interactive=False)

                  def display_mantenimiento(selected_option):
                       if not selected_option:
                            return "Error: Seleccione un equipo."
                       equipo_id = selected_option.split(' - ')[0]
                       return gestor.get_mantenimiento(equipo_id)

                  # Trigger display when the dropdown value changes
                  maint_equipo_dropdown_view.change(
                       display_mantenimiento,
                       inputs=maint_equipo_dropdown_view,
                       outputs=ver_maint_output
                  )

             with gr.Accordion("Eliminar Mantenimiento"):
                 # Ensure the maint_equipo_dropdown_del is within this accordion
                 gr.Markdown("Seleccione el equipo y el registro de mantenimiento a eliminar:")
                 maint_equipo_dropdown_del.render()
                 maint_record_dropdown_del = gr.Dropdown(label="Seleccione el registro de mantenimiento:", choices=[], interactive=True)
                 btn_eliminar_maint = gr.Button("Eliminar Mantenimiento Seleccionado")
                 maint_del_output = gr.Textbox(label="Estado", interactive=False)

                 def load_maint_records_for_delete(selected_equipo_option):
                      if not selected_equipo_option:
                          return gr.Dropdown(choices=[], value=None, interactive=True)
                      equipo_id = selected_equipo_option.split(' - ')[0]
                      options = gestor.get_mantenimiento_options(equipo_id)
                      return gr.Dropdown(choices=options, value=options[0] if options else None, interactive=True)

                 maint_equipo_dropdown_del.change(
                      load_maint_records_for_delete,
                      inputs=maint_equipo_dropdown_del,
                      outputs=maint_record_dropdown_del
                 )

                 def delete_mantenimiento(selected_equipo_option, selected_maint_option):
                      if not selected_equipo_option:
                          return "Error: Seleccione un equipo.", gr.Dropdown(choices=[], value=None, interactive=True)
                      if not selected_maint_option:
                           return "Error: Seleccione un registro de mantenimiento.", gr.Dropdown(choices=[], value=None, interactive=True)

                      equipo_id = selected_equipo_option.split(' - ')[0]
                      try:
                          record_index_str = selected_maint_option.split(':')[0]
                          record_index = int(record_index_str) - 1
                      except (ValueError, IndexError):
                           return "Error: Formato de registro de mantenimiento inválido.", gr.Dropdown(choices=[], value=None, interactive=True)

                      result = gestor.eliminar_mantenimiento(equipo_id, record_index)
                      new_options = gestor.get_mantenimiento_options(equipo_id)
                      return result, gr.Dropdown(choices=new_options, value=new_options[0] if new_options else None, interactive=True)

                 btn_eliminar_maint.click(
                      delete_mantenimiento,
                      inputs=[maint_equipo_dropdown_del, maint_record_dropdown_del],
                      outputs=[maint_del_output, maint_record_dropdown_del]
                 )

        # Calibraciones Tab
        with gr.TabItem("GESTION DE CALIBRACIONES"):
             with gr.Accordion("Programar Calibración"):
                 # Ensure the calib_equipo_dropdown_reg is within this accordion
                 gr.Markdown("Seleccione el equipo para programar calibración:")
                 calib_equipo_dropdown_reg.render()
                 calib_fecha = gr.Textbox(label="Fecha Programada (YYYY-MM-DD)", placeholder="YYYY-MM-DD")
                 calib_desc = gr.Textbox(label="Descripción")
                 calib_tec = gr.Textbox(label="Técnico responsable")
                 btn_registrar_calib = gr.Button("Programar Calibración")
                 calib_reg_output = gr.Textbox(label="Estado", interactive=False)

                 def register_calibracion(selected_option, fecha_str, desc, tec):
                      if not selected_option:
                          return "Error: Seleccione un equipo."
                      if not fecha_str:
                           return "Error: Ingrese una fecha."
                      equipo_id = selected_option.split(' - ')[0]
                      result = gestor.registrar_calibracion(equipo_id, fecha_str, desc, tec)
                      return result

                 btn_registrar_calib.click(
                      register_calibracion,
                      inputs=[calib_equipo_dropdown_reg, calib_fecha, calib_desc, calib_tec],
                      outputs=calib_reg_output
                 )

             with gr.Accordion("Historial De Calibración"):
                  # Use calib_equipo_dropdown_view here
                  gr.Markdown("Seleccione el equipo para ver el historial de calibración:")
                  calib_equipo_dropdown_view.render()
                  ver_calib_output = gr.TextArea(label="Historial De Calibración", interactive=False)

                  def display_calibracion(selected_option):
                       if not selected_option:
                            return "Error: Seleccione un equipo."
                       equipo_id = selected_option.split(' - ')[0]
                       return gestor.get_calibracion(equipo_id)

                  # Trigger display when the dropdown value changes
                  calib_equipo_dropdown_view.change(
                       display_calibracion,
                       inputs=calib_equipo_dropdown_view,
                       outputs=ver_calib_output
                  )

             with gr.Accordion("Eliminar Calibración"):
                  # Ensure the calib_equipo_dropdown_del is within this accordion
                  gr.Markdown("Seleccione el equipo y el registro de calibración a eliminar:")
                  calib_equipo_dropdown_del.render()
                  calib_record_dropdown_del = gr.Dropdown(label="Seleccione el registro de calibración:", choices=[], interactive=True)
                  btn_eliminar_calib = gr.Button("Eliminar Calibración Programada")
                  calib_del_output = gr.Textbox(label="Estado", interactive=False)

                  def load_calib_records_for_delete(selected_equipo_option):
                      if not selected_equipo_option:
                          return gr.Dropdown(choices=[], value=None, interactive=True)
                      equipo_id = selected_equipo_option.split(' - ')[0]
                      options = gestor.get_calibracion_options(equipo_id)
                      return gr.Dropdown(choices=options, value=options[0] if options else None, interactive=True)

                  calib_equipo_dropdown_del.change(
                       load_calib_records_for_delete,
                       inputs=calib_equipo_dropdown_del,
                       outputs=calib_record_dropdown_del
                  )

                  def delete_calibracion(selected_equipo_option, selected_calib_option):
                       if not selected_equipo_option:
                           return "Error: Seleccione un equipo.", gr.Dropdown(choices=[], value=None, interactive=True)
                       if not selected_calib_option:
                            return "Error: Seleccione un registro de calibración.", gr.Dropdown(choices=[], value=None, interactive=True)

                       equipo_id = selected_equipo_option.split(' - ')[0]
                       try:
                            record_index_str = selected_calib_option.split(':')[0]
                            record_index = int(record_index_str) - 1
                       except (ValueError, IndexError):
                            return "Error: Formato de registro de calibración inválido.", gr.Dropdown(choices=[], value=None, interactive=True)

                       result = gestor.eliminar_calibracion(equipo_id, record_index)
                       new_options = gestor.get_calibracion_options(equipo_id)
                       return result, gr.Dropdown(choices=new_options, value=new_options[0] if new_options else None, interactive=True)

                  btn_eliminar_calib.click(
                       delete_calibracion,
                       inputs=[calib_equipo_dropdown_del, calib_record_dropdown_del],
                       outputs=[calib_del_output, calib_record_dropdown_del]
                  )


        # Alerts Section (separate tab)
        with gr.TabItem("Alertas"):
             btn_check_alerts = gr.Button("Ver Alertas")
             alerts_output = gr.TextArea(label="Alertas de Próximos Eventos", interactive=False)

             def check_alerts():
                 return gestor.check_alerts()

             btn_check_alerts.click(check_alerts, outputs=alerts_output)

    # Save Data Button (outside tabs for easy access)
    with gr.Row():
        btn_save_data = gr.Button("Guardar Datos")
        save_output = gr.Textbox(label="Estado de Guardado", interactive=False)

    def save_data():
        guardar_datos(gestor.equipos, gestor.mantenimiento, gestor.calibracion, gestor.alertas)
        return "Datos guardados exitosamente!"

    btn_save_data.click(save_data, outputs=save_output)

    # Register the save_data function to be called when the Gradio app is closed
    # Note: Auto-saving on close using browser events is not fully reliable.
    # A dedicated save button is the recommended approach.
    app.load(
        None,
        None,
        js="""
        function() {
            const saveButton = document.querySelector('button[data-testid="save_data"]');
            if (saveButton) {
                window.addEventListener('beforeunload', (event) => {
                    saveButton.click();
                });
            } else {
                console.log("Save button not found. Auto-save on close will not work.");
            }
        }
        """
    )

# --- Main Execution Block ---
if __name__ == "__main__":
    app.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80180792dbb06b1e37.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
